In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [4]:
# Load ANN train model, scaler model,pickle model
model=load_model('model.h5')

with open('label_encoder_gen.pkl','rb') as file:
    label_encode_gen=pickle.load(file)


with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)


with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [3]:
input_data={
  "CreditScore": 500,
  "Geography": "France",
  "Gender": "Male",
  "Age": 40,
  "Tenure": 3,
  "Balance": 60000,
  "NumOfProducts": 2,
  "HasCrCard": 1,
  "IsActiveMember": 1,
  "EstimatedSalary": 50000,
}

In [11]:
encoder_geo=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(encoder_geo,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\PythonTutorial\udemy\GenAICourse\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
input_data=pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,Male,40,3,60000,2,1,1,50000


In [17]:
input_df=pd.concat([input_data.reset_index(drop=True), geo_encoded_df],axis=1)

In [18]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,500,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0


In [21]:
input_df['Gender']=label_encode_gen.transform(input_data['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,500,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0


In [24]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [26]:
input_df = input_df.drop('Geography', axis=1).reset_index(drop=True)
geo_encoded_df = geo_encoded_df.reset_index(drop=True)

input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,500,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [28]:
scaler.feature_names_in_


array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [30]:
input_df = input_df.loc[:, ~input_df.columns.duplicated()]


In [31]:
input_df.columns


Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [33]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-1.55709758,  0.90793278,  0.0999528 , -0.68930551, -0.27575413,
         0.80581296,  0.65131528,  0.968496  , -0.88740526,  1.00573071,
        -0.58614849, -0.57295135]])

In [35]:
#prediction
prediction=model.predict(input_scaled)

1/1 [==============================] - 0s 43ms/step


In [36]:
predict_prob=prediction[0][0]
predict_prob

0.03323773

In [37]:
if predict_prob>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
